# Imports and Data needed (Please Run This!!)

**The data needed for the graphs are:**


*   avg_city_SH_monthly.json
*   monthly_avg.json
*   meals_SH_canteens_merged_filtered_.csv




**Imports:**

In [3]:
import json
import pandas as pd
import datetime
import numpy as np
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
from plotly.subplots import make_subplots

**Define here where the path to the actual Data is. It will find the data needed in that path:**

In [36]:
file_path_data = ""

# Cleaning Data Set for Visualization

## Dietary Options - Average Price Data (monthly_avg.json)


In [8]:
df = pd.read_csv('meals_SH_canteens_merged_filtered_.csv', sep='@')

df['average_price'] = df[['student_price', 'employee_price', 'guest_price']].mean(axis=1)
# excluding -1 in 'vvo_status' (-1 means not defined category)
category_df = df[df['vvo_status'] != '-1'].copy()
average_prices_category = category_df.groupby(['date', 'vvo_status'])['average_price'].mean().unstack(fill_value=0)
average_prices_all = df.groupby('date')['average_price'].mean()
average_prices_category['all'] = average_prices_all

# Reorder
average_prices_category.reset_index(inplace=True)
average_prices_category = average_prices_category[['date', 'all'] + [col for col in average_prices_category.columns if col not in ('date', 'all')]]

average_prices_category

vvo_status,date,all,meat,vegan,veget.
0,2021-08-27,3.805556,4.083333,0.000000,0.000000
1,2021-08-30,3.955556,3.891667,0.000000,0.000000
2,2021-08-31,3.472222,3.983333,0.000000,3.100000
3,2021-09-01,4.055556,4.066667,0.000000,0.000000
4,2021-09-02,3.838889,5.033333,0.000000,0.000000
...,...,...,...,...,...
745,2024-02-26,4.244872,4.416667,4.264583,3.756250
746,2024-02-27,4.667014,5.160714,4.600000,3.645833
747,2024-02-28,4.303261,4.303125,4.392029,4.011905
748,2024-02-29,4.393262,4.493137,4.754902,3.972222


In [9]:
average_price_category_json = average_prices_category.to_json(orient='records')
json_file_path = 'average_price_category.json'

with open(json_file_path, 'w') as file:
    file.write(average_price_category_json)

json_file_path

'average_price_category.json'

In [10]:
df = pd.read_json('average_price_category.json')

df['date'] = pd.to_datetime(df['date'])

# Replace 0 with NaN to avoid mistakes
df.replace(0, np.nan, inplace=True)
df.set_index('date', inplace=True)
df_monthly_avg = df.resample('M').mean()
df_monthly_avg.reset_index(inplace=True)

In [11]:
average_price_category_json = df_monthly_avg.to_json(orient='records')
json_file_path = 'monthly_avg.json'

with open(json_file_path, 'w') as file:
    file.write(average_price_category_json)

json_file_path

'monthly_avg.json'

## Average Price of Cities (monthly_avg.json)

In [16]:
df = pd.read_csv('meals_SH_canteens_merged_filtered_.csv', sep='@')

with open('german_canteens.json', 'r') as file:
    german_canteens = json.load(file)

canteens_df = pd.DataFrame(german_canteens)
canteens_df.rename(columns={'id': 'mensa_id'}, inplace=True)
df = pd.merge(df, canteens_df[['mensa_id', 'city']], on='mensa_id', how='left')
df['average_price'] = df[['student_price', 'employee_price', 'guest_price']].mean(axis=1)
avg_city_SH = df.groupby(['date', 'city'])['average_price'].mean().reset_index()

In [17]:
avg_city_SH_file_path = 'avg_city_SH.json'
avg_city_SH.to_json(avg_city_SH_file_path, orient='records')
avg_city_SH_file_path

'avg_city_SH.json'

In [18]:
# Convert to datetime
df['date'] = pd.to_datetime(df['date'])
monthly_avg_price_per_city = df.set_index('date').groupby('city').resample('M')['average_price'].mean().reset_index()

In [19]:
monthly_avg_price_per_city_file_path = 'avg_city_SH_monthly.json'
monthly_avg_price_per_city.to_json(monthly_avg_price_per_city_file_path, orient='records')

monthly_avg_price_per_city_file_path

'avg_city_SH_monthly.json'

# Visualization 1: Price Over Time by City

## Graph

In [ ]:
df_monthly = pd.read_json(f'{file_path_data}avg_city_SH_monthly.json')
df_monthly['date'] = pd.to_datetime(df_monthly['date'], unit='ms')
df_monthly_clean = df_monthly.dropna(subset=['average_price'])
cities = df_monthly_clean['city'].unique()

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
color_idx = 0
fig1 = go.Figure()
city_trace_indices = {city: [] for city in cities}

for city in cities:
    city_data = df_monthly_clean[df_monthly_clean['city'] == city]
    if city_data.empty:
        continue
    X = city_data['date'].map(datetime.datetime.toordinal).values.reshape(-1, 1)
    y = city_data['average_price'].values

    # Linear regression
    reg = LinearRegression().fit(X, y)
    y_pred = reg.predict(X)

    # Plot data
    actual_trace = go.Scatter(x=city_data['date'], y=y, mode='markers+lines', name=city, marker_color=colors[color_idx])
    fig1.add_trace(actual_trace)
    city_trace_indices[city].append(len(fig1.data) - 1)

    # Plot trend line
    trend_trace = go.Scatter(x=city_data['date'], y=y_pred, mode='lines', name=f"{city} Trend", line=dict(color=colors[color_idx], dash='dash'))
    fig1.add_trace(trend_trace)
    city_trace_indices[city].append(len(fig1.data) - 1)

    color_idx = (color_idx + 1) % len(colors)

# buttons for dropdown
buttons = [
    dict(label='All',
         method='update',
         args=[{'visible': [True] * len(fig1.data)},
               {'title': 'Monthly Average Prices by City'}])
]

for city, indices in city_trace_indices.items():
    visible = [False] * len(fig1.data)
    for index in indices:
        visible[index] = True
    button = dict(label=city,
                  method='update',
                  args=[{'visible': visible},
                        {'title': f'Monthly Average Prices - {city}'}])
    buttons.append(button)

# Add dropdown
fig1.update_layout(
    updatemenus=[dict(buttons=buttons,
                      direction="down",
                      pad={"r": 10, "t": 10},
                      showactive=True,
                      x=0,
                      xanchor="left",
                      y=1.15,
                      yanchor="top")],
    title='Monthly Average Prices by City',
    xaxis_title='Date',
    yaxis_title='Average Price (in €)',
    legend=dict(title="City", orientation="v", x=1, xanchor="left", y=1, yanchor="auto")
)

# fig1.show()


## Calculations

### Average Price Change in € and %



Average Price Change in €

In [20]:
df_monthly_prices = pd.read_json('avg_city_SH_monthly.json')

# Convert to datetime
df_monthly_prices['date'] = pd.to_datetime(df_monthly_prices['date'], unit='ms')
df_monthly_prices_sorted = df_monthly_prices.sort_values(by=['city', 'date'])

# Calculate the price change for each city
df_monthly_prices_sorted['price_change'] = df_monthly_prices_sorted.groupby('city')['average_price'].diff()
average_price_change_per_city = df_monthly_prices_sorted.groupby('city')['price_change'].mean().reset_index()

average_price_change_per_city

,city,price_change
0,Flensburg,0.034873
1,Heide,0.038874
2,Kiel,0.034415
3,Lübeck,0.013045


Average Price Change in %

In [21]:
df_monthly_prices_sorted['percentage_change'] = df_monthly_prices_sorted.groupby('city')['average_price'].pct_change() * 100
average_percentage_change_per_city = df_monthly_prices_sorted.groupby('city')['percentage_change'].mean().reset_index()

average_percentage_change_per_city

,city,percentage_change
0,Flensburg,0.920610
1,Heide,0.779610
2,Kiel,0.868170
3,Lübeck,0.722452


### Price Change 2022 and 2023

In [59]:
city_names = ["Kiel", "Flensburg", "Lübeck", "Heide"]

In [60]:
from datetime import datetime

# Load the updated data from the uploaded file
with open('avg_city_SH_monthly.json', 'r') as file:
    updated_data = json.load(file)

for city_name in city_names:
  city_data = [d for d in updated_data if d['city'] == city_name]
  for entry in city_data:
      entry['date'] = datetime.utcfromtimestamp(entry['date'] / 1000).strftime('%Y-%m')
  city_data_filtered = [d for d in city_data if d['average_price'] is not None]

  # Calculate for 2022
  city_prices_2022 = [d['average_price'] for d in city_data_filtered if '2022' in d['date']]
  price_changes_2022 = [city_prices_2022[i+1] - city_prices_2022[i] for i in range(len(city_prices_2022)-1)]
  average_price_change_2022 = sum(price_changes_2022) / len(price_changes_2022) if price_changes_2022 else 0

  percent_changes_2022 = [(price_changes_2022[i] / city_prices_2022[i]) * 100 for i in range(len(price_changes_2022))] if price_changes_2022 else [0]
  average_percent_change_2022 = sum(percent_changes_2022) / len(percent_changes_2022) if percent_changes_2022 else 0

  # Calculate for 2023
  city_prices_2023 = [d['average_price'] for d in city_data_filtered if '2023' in d['date']]
  price_changes_2023 = [city_prices_2023[i+1] - city_prices_2023[i] for i in range(len(city_prices_2023)-1)]
  average_price_change_2023 = sum(price_changes_2023) / len(price_changes_2023) if price_changes_2023 else 0

  percent_changes_2023 = [(price_changes_2023[i] / city_prices_2023[i]) * 100 for i in range(len(price_changes_2023))] if price_changes_2023 else [0]
  average_percent_change_2023 = sum(percent_changes_2023) / len(percent_changes_2023) if percent_changes_2023 else 0

  print(city_name)
  print(f"2022: {average_price_change_2022} €, or {average_percent_change_2022} %")
  print(f"2023: {average_price_change_2023} €, or {average_percent_change_2023} %")


Kiel
2022: 0.029587814436363612 €, or 0.822677208431401 %
2023: 0.04501181756363636 €, or 1.089611637641197 %
Flensburg
2022: 0.005749601272727268 €, or 0.2893313003093822 %
2023: 0.03566621730000003 €, or 0.9709508745361279 %
Lübeck
2022: 0.05211863245454545 €, or 1.5517245271120914 %
2023: 0.023322670818181814 €, or 1.2328595894344152 %
Heide
2022: 0.045809116809999974 €, or 1.2436370185536147 %
2023: 0.053625429550000006 €, or 1.33508842292582 %


# Visualization 2: Price Over Time by Status (student/employee/guest)

## Graph

In [37]:
df = pd.read_csv(f'{file_path_data}meals_SH_canteens_merged_filtered_.csv', sep='@')
df['date'] = pd.to_datetime(df['date'])
df['student_price'] = pd.to_numeric(df['student_price'], errors='coerce')
df['employee_price'] = pd.to_numeric(df['employee_price'], errors='coerce')
df['guest_price'] = pd.to_numeric(df['guest_price'], errors='coerce')

# Calculate average prices by month
average_prices_by_month = df.groupby(df['date'].dt.to_period('M')).agg({
    'student_price': 'mean',
    'employee_price': 'mean',
    'guest_price': 'mean'
}).reset_index()

average_prices_by_month['date'] = average_prices_by_month['date'].dt.to_timestamp()

fig2 = go.Figure()
categories = ['student_price', 'employee_price', 'guest_price']
category_labels = ['Students', 'Employees', 'Guests']
colors = ['blue', 'red', 'green']

for category, label, color in zip(categories, category_labels, colors):
    x = average_prices_by_month['date']
    y = average_prices_by_month[category]

    # Plot
    fig2.add_trace(go.Scatter(x=x, y=y, mode='markers+lines', name=label, marker_color=color))

    # Linear regression / trend line
    x_ordinal = np.array([i.toordinal() for i in x])
    x_ordinal = x_ordinal.reshape(-1, 1)
    y_values = y.values.reshape(-1, 1)
    model = LinearRegression().fit(x_ordinal, y_values)

    # Plot
    y_pred = model.predict(x_ordinal)
    fig2.add_trace(go.Scatter(x=x, y=y_pred.flatten(), mode='lines', name=f'{label} Trend', line=dict(color=color, dash='dash')))

fig2.update_layout(title='Average Meal Prices in Schleswig-Holstein',
                  xaxis_title='Date',
                  yaxis_title='Average Price (€)',
                  legend_title="Category",
                  xaxis=dict(tickangle=-45))

# Make graph interactive
fig2.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(label="All",
                     method="update",
                     args=[{"visible": [True, True, True, True, True, True]},
                           {"title": "All Categories"}]),
                dict(label="Students",
                     method="update",
                     args=[{"visible": [True, True, False, False, False, False]},
                           {"title": "Average Prices for Students"}]),
                dict(label="Employees",
                     method="update",
                     args=[{"visible": [False, False, True, True, False, False]},
                           {"title": "Average Prices for Employees"}]),
                dict(label="Guests",
                     method="update",
                     args=[{"visible": [False, False, False, False, True, True]},
                           {"title": "Average Prices for Guests"}]),
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0,
            xanchor="left",
            y=1.15,
            yanchor="top"
        ),
    ]
)

# fig2.show()

## Calculations (run Graph for this first)

In [40]:
average_price_changes = {}
percentage_changes = {}

for category in categories:
    # Price change
    average_price_changes[category] = average_prices_by_month[category].diff().mean()

    # Percentage change
    percentage_changes[category] = average_prices_by_month[category].pct_change().mean() * 100

print(f"In € {average_price_changes} \nIn % {percentage_changes}")

In € {'student_price': 0.025768049155145925, 'employee_price': 0.03502304147465438, 'guest_price': 0.039144905273937505} 
In % {'student_price': 0.9113037895647127, 'employee_price': 0.8727065204908688, 'guest_price': 0.9102749329938669}


# Visualization 3: Price Over Time by Category

## Graph

In [74]:
with open(f'{file_path_data}monthly_avg.json', 'r') as file:
    monthly_data = json.load(file)

# DataFrame
df_monthly = pd.DataFrame(monthly_data)
df_monthly['date'] = pd.to_datetime(df_monthly['date'], unit='ms')

colors = {'all': 'red', 'vegan': 'green', 'veget.': 'blue', 'meat': 'purple'}
fig3 = go.Figure()

trace_names = ['all', 'vegan', 'veget.', 'meat']
trace_visibility = {
    'All': [True] * 8,
    'Overall': [True, True] + [False] * 6,
    'Vegan': [False] * 2 + [True, True] + [False] * 4,
    'Veget.': [False] * 4 + [True, True] + [False] * 2,
    'Meat': [False] * 6 + [True, True],
}

for category in trace_names:
    display_name = 'overall' if category == 'all' else category

    fig3.add_trace(
        go.Scatter(x=df_monthly['date'], y=df_monthly[category], mode='markers+lines', name=display_name, marker_color=colors[category]),
    )

    # Linear regression
    df_non_null = df_monthly.dropna(subset=[category])
    X = np.array(range(len(df_non_null))).reshape(-1, 1)
    y = df_non_null[category].values.reshape(-1, 1)
    model = LinearRegression().fit(X, y)
    trend_line = model.predict(np.array(range(len(df_monthly))).reshape(-1, 1))

    fig3.add_trace(
        go.Scatter(x=df_monthly['date'], y=trend_line.ravel(), mode='lines', name=f'{display_name} trend', line=dict(color=colors[category], dash='dot')),
    )

# Dropdown menu
buttons = []
for label, visibility in trace_visibility.items():
    button = dict(
        label=label,
        method="update",
        args=[{"visible": visibility},
              {"title": f"Monthly Average Prices by Category - {label}"}])
    buttons.append(button)

fig3.update_layout(
    updatemenus=[dict(
        buttons=buttons,
        direction="down",
        pad={"r": 10, "t": 10},
        showactive=True,
        x=0,
        xanchor="left",
        y=1.15,
        yanchor="top"
    )],
    title='Monthly Average Prices by Category',
    xaxis_title='Date (per Month)',
    yaxis_title='Price (in €)',
    legend=dict(title="Category", orientation="v", x=1, xanchor="left", y=1, yanchor="auto")
)

# fig3.show()


## Calculations

In [72]:
file_path = 'monthly_avg.json'
with open(file_path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'], unit='ms')

months_span = ((df['date'].iloc[-1] - df['date'].iloc[0]) / pd.Timedelta(days=1)) / 30.44 # Avr Days in Month
changes_eur_per_month = {}
changes_percent_per_month = {}

categories = ['meat', 'vegan', 'veget.', 'all']

for category in categories:
    # Calculate average monthly percentage change
    total_change = df[category].iloc[-1] - df[category].iloc[0]
    avg_monthly_change_eur = total_change / months_span
    percent_change = (total_change / df[category].iloc[0]) * 100
    avg_monthly_percent_change = percent_change / months_span

    changes_eur_per_month[category] = avg_monthly_change_eur
    changes_percent_per_month[category] = avg_monthly_percent_change

print("Average Monthly Price Changes in €:", changes_eur_per_month)
print("Average Monthly Price Changes in %:", changes_percent_per_month)

Average Monthly Price Changes in €: {'meat': 0.05592950983893532, 'vegan': 0.04514280334701167, 'veget.': 0.03981194768361823, 'all': 0.03803749387287382}
Average Monthly Price Changes in %: {'meat': 1.4031096544998494, 'vegan': 1.528037211759526, 'veget.': 1.2842563768909105, 'all': 1.015838115284229}


The following calculations were made before and could easily be replaced by simply multiplying the values above by 12.

In [73]:
file_path = 'monthly_avg.json'
with open(file_path, 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'], unit='ms')

df.dropna(subset=['meat', 'vegan', 'veget.'], inplace=True)
days_span = (df['date'].iloc[-1] - df['date'].iloc[0]).days
years_span = days_span / 365.25  # Avr Days in Year
average_price_changes_eur = {}
average_price_changes_percent = {}

categories = ['meat', 'vegan', 'veget.']

for category in categories:
    # Calculate average monthly percentage change
    total_price_change = df[category].iloc[-1] - df[category].iloc[0]
    avg_yearly_change_eur = total_price_change / years_span
    total_percent_change = (total_price_change / df[category].iloc[0]) * 100
    avg_yearly_percent_change = total_percent_change / years_span

    average_price_changes_eur[category] = avg_yearly_change_eur
    average_price_changes_percent[category] = avg_yearly_percent_change

print("Average Yearly Price Changes in €:", average_price_changes_eur)
print("Average Yearly Price Changes in %:", average_price_changes_percent)

Average Yearly Price Changes in €: {'meat': 0.6710989969997083, 'vegan': 0.5416691498848887, 'veget.': 0.4777041357240985}
Average Yearly Price Changes in %: {'meat': 16.835933025823586, 'vegan': 18.334940591168422, 'veget.': 15.409810829809627}
